<a href="https://colab.research.google.com/github/CianOSull/AutoML_With_SA_FYP_2021/blob/AutoKeras/Copy_of_AutoKeras_MainNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generic Notebook for running all the libraries

How this notebook works is that it contains the code
for loading and cleaning the dataset.

Then there is multiple branches created on the
Github that include the code for running each library.

E.g. MLBox branch has the code for running MLBox.

# CURRENT BRANCH = AUTOKERAS

# Install the necessary library
Run the install code in the code cell below.

In [21]:
# Insert any install comamnds in this cell
!pip3 install autokeras

# Preprocessing Section

In [22]:
# Import the necessary modules for cleaning
import math
import time 
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize 
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [23]:
# Create the set of stopwords for cleaning text
stopwords = set(w.rstrip() for w in open('/content/drive/MyDrive/CIT/FYP/ImplementationFiles/stopwords.txt'))

In [24]:
# Download the necessary parts for the NLTK module
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [25]:
# This funciton handles celaning text
def clean_text(text):
    # Create the lemmatizer
    wordnet_lemmatizer = WordNetLemmatizer()
    
    # Get rid of non alpha characters except "'" as it is needed for the lemment
    text = "".join(c for c in text if c.isalnum() or c == " " or "'")
    
    # Get rid of capitals
    text = text.lower()
    
    # Tokenize the words    
    # Create tokens of each word
    token_text = word_tokenize(text)
    
    # Get rid of any piece of text that isn't over 2 characters
    token_text = [t for t in token_text if len(t) > 2] 
    
    # Put words in base form by doing lemmatization
    token_text = [wordnet_lemmatizer.lemmatize(t) for t in token_text]

    # Remove stopwords
    token_text = [t for t in token_text if t not in stopwords]
    
    # Return the tokens
    return token_text

In [26]:
# This function will get the term frequencies for word in the review
# TF = Term I frequency in document/total words in document
def calc_tf(term_count, review_corpus):
    # A dictionary of all the term frequencies found
    tf_freq = dict.fromkeys(term_count.keys(), 0)   
    
    # Review corpus is a tokenized list so the total words iteh length
    total_words = len(review_corpus)
    
    # Calculate the term frequency for each word
    for word, count in term_count.items():
        tf_freq[word] = count/total_words
        
    return tf_freq

In [27]:
# This calcualtes the idf
# IDF = log(2)*(Total number of Documents/documents frequency or documents with term)
def calc_idf(unique_terms, list_doc_terms):   
    # A dicitonary of all the inverse document frequencies
    idf = dict.fromkeys(unique_terms, 0)
    
    # Basically list_doc_terms has all the documents with the term count for each word
    # You go through each document count the terms where they occured
    for doc_terms in list_doc_terms:  
        # This for loop is counting the amount of document a word was in
        for word, value in doc_terms.items():
            if 0 < value:
                idf[word] += 1
        
    # Now we calculate idf
    for word, value in idf.items():
        idf[word] = math.log10(10 / float(value))
    
    return idf

In [28]:
# Modified this function to return a list as dictionaries arn't needed anymore
def calc_tf_idf(tf, idf, n_terms):
    # Create an array that is of length of the number of unique terms
    tf_idf_array = np.zeros(n_terms)
    
    for index, (word, value) in enumerate(tf.items()):
        # Add the tfidf to the array
        tf_idf_array[index] = value*idf[word]
    
    return tf_idf_array

In [29]:
def process_text(text_data):
    # A list of all the cleaned reviews
    doc_list = []
    
    # List of all the unique terms
    unique_terms = []
    
    # A list of all the term frequencies
    tf_list = []
    
    for review in text_data:
        # First clean the review
        clean_review = clean_text(review)
        
        # Keeps track of the term counts for each word
        count_dict = {}
        
        # Now lets find the total count for each word
        for token in clean_review:
            if token not in count_dict:
                count_dict[token] = 1
            else:
                count_dict[token] += 1
        
        # Caclulate the term frequencies for each document
        tf_list.append(calc_tf(count_dict, clean_review))
        
        # Then add the dictionary of counts for each document to the list
        doc_list.append(count_dict)
        
        # Then add the new unique terms
        unique_terms = set(unique_terms).union(set(clean_review))
    
    # Calculate the inverse document frequency value
    idf = calc_idf(unique_terms, doc_list)
    
    # This array will contain the tfidf values for each term in each review
    tfidf_values = np.zeros((len(tf_list), len(unique_terms)))
    
    # Now we can get the TFIDF for each document
    for index, term_freq in enumerate(tf_list):
        # This will return an array of the tfidf values calculated.
        # The length of the unique terms list is passed in so that the 
        # Array that is returned matches the tfidf array
        tf_idf_array = calc_tf_idf(term_freq, idf, len(unique_terms))
        # Add this to the overall tfidf values calculated
        tfidf_values[index,:] = tf_idf_array
    
    return tfidf_values

In [30]:
# Prepare the data
def prepare_data():
    print("="*50)

    # Load the dataset
    # imdb_df = pd.read_csv("IMDB Dataset.csv")
    imdb_df = pd.read_csv("/content/drive/MyDrive/CIT/FYP/ImplementationFiles/IMDB_Dataset.csv")
    print("Dataset loaded")
    print("="*50)

    
    # Change each positive and negative value to 1 and 0 respectively    
    imdb_df['sentiment'] = imdb_df['sentiment'].map({'negative' : 0, 'positive' : 1})
    
    # For testing, a much smaller dataset is going to be used
    imdb_df = imdb_df.head(5000)

    # Group all the negative reviews and get the first 2500
    imdb_df_neg = (imdb_df[imdb_df['sentiment'] == 0])[0:2500]
    # Group all the positive and get the first 2500
    imdb_df_pos = imdb_df[imdb_df['sentiment'] == 1]
    
    test_df = pd.concat([imdb_df_neg, imdb_df_pos]) 
    # print(test_df)
    
    # .values on a column of a dataframe returns a numpy array
    # This is a numpy array of all the reviews
    # initial_reviews = imdb_df['review'].values
    initial_reviews = test_df['review'].values
    
    # This is a numpy array of all the positive and negativelabels
    # labels = imdb_df['sentiment'].values
    labels = test_df['sentiment'].values
    
    print("Creating Feature Vector")
    print("="*50)
    start = time.time()
    # Process the text data and create teh feature vector
    feature_vector = process_text(initial_reviews)
    end = time.time()
    print("Feature Vector Created")
    print(f"Execution time is {end - start} secs")
    print("="*50)
    
    # Shuffle the labesl and feature vector using sklearn shuffle
    feature_vector, labels = shuffle(feature_vector, labels)
    
    # Creating train and test data
    # The splits will be 80:20 
    no_samples = 0.8
    
    # This gets the percentage of indexes from feature vector and uses those for training
    X_train = feature_vector[0:int(no_samples*len(feature_vector))]
    y_train = labels[0:int(no_samples*len(labels))]
    
    # Go from the index that was used for training to the final
    X_test = feature_vector[int(no_samples*len(feature_vector)):len(feature_vector)]
    y_test = labels[int(no_samples*len(labels)):len(labels)]

    return X_train, y_train, X_test, y_test

# Create Model Section 

**Exporting models:**

https://autokeras.com/tutorial/export/

Auto Keras has a built in text classifier that takes in strings so going to test using that?



Look into checking the validation accuracy in autokeras
Early stopping based on the accuracy and loss for the validation to stop once it platous
Validation shows when training is not learning anymore


In [31]:
import autokeras as ak

In [60]:
# Load the dataset
imdb_df = pd.read_csv("/content/drive/MyDrive/CIT/FYP/ImplementationFiles/IMDB_Dataset.csv")
# print(imdb_df.head())

# For testing, a much smaller dataset is going to be used
# imdb_df = imdb_df.head(4000)
# print(imdb_df.head())

imdb_df['sentiment'] = imdb_df['sentiment'].map({'negative' : 0, 'positive' : 1})

# Group all the negative reviews and get the first 25000
# imdb_df_neg = (imdb_df[imdb_df['sentiment'] == "negative"])[0:2500]
imdb_df_neg = (imdb_df[imdb_df['sentiment'] == 0])[0:2500]
# Group all the positive and get the first 25000
# imdb_df_pos = imdb_df[imdb_df['sentiment'] == "positive"][0:2500]
imdb_df_pos = imdb_df[imdb_df['sentiment'] == 1][0:2500]

# Combine the two split positives and negatives into one dataframe
imdb_df = pd.concat([imdb_df_neg, imdb_df_pos]) 

# .values on a column of a dataframe returns a numpy array
# This is a numpy array of all the reviews
# initial_reviews = imdb_df['review'].values
feature_vector = imdb_df['review'].values
# print(feature_vector)

# This is a numpy array of all the positive and negativelabels
labels = imdb_df['sentiment'].values

# Shuffle the labesl and feature vector using sklearn shuffle
feature_vector, labels = shuffle(feature_vector, labels)

# Creating train and test data
# The splits will be 80:20 
no_samples = 0.8

# This gets the percentage of indexes from feature vector and uses those for training
X_train = feature_vector[0:int(no_samples*len(feature_vector))]
y_train = labels[0:int(no_samples*len(labels))]

# Go from the index that was used for training to the final
X_test = feature_vector[int(no_samples*len(feature_vector)):len(feature_vector)]
y_test = labels[int(no_samples*len(labels)):len(labels)]

print(X_train.shape)
print(type(X_train))

(4000,)
<class 'numpy.ndarray'>


In [64]:
# Create the text classifier
# It is only going to do 1 model but change max trials for more
# Has a built in project name, default is text_classifier
# Trials is the amount of keras models to use, default is 1000 and might stop before reaching that number
# Directory sets where search outputs will be saved
# Objective is what it needs to prioritise, default is val_loss but val_accuracy can be used
# Naming convention = ak_epochs_dataNo_trials
clf = ak.TextClassifier(overwrite=True, project_name="ak_10_5000_2", max_trials=2, directory="/content/drive/MyDrive/CIT/FYP/ImplementationFiles/ExportedModels/AutoKeras/SearchOuputs", objective="val_loss")

In [65]:
# Call the fit function and train the models for only 2 epochs
# Leave empty for adaptive epochs
# By default, auto keras will use the last 20% of training data for validation
# Setting validation to 0.2 in this case to just to demonstrate
print("Creating Fit the model")
print("="*50)
start = time.time()
# Default epochs is 1000
clf.fit(X_train, y_train, validation_split=0.2, epochs=10)
end = time.time()
print("Model Fitting Finished")
print(f"Execution time is {end - start} secs")
print("="*50)

Trial 2 Complete [00h 01m 40s]
val_loss: 0.4093758463859558

Best val_loss So Far: 0.39874398708343506
Total elapsed time: 00h 05m 20s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
125/125 [==============================] - 26s 205ms/step - loss: 0.6956 - accuracy: 0.5109
Epoch 2/10
125/125 [==============================] - 25s 203ms/step - loss: 0.5897 - accuracy: 0.6956
Epoch 3/10
125/125 [==============================] - 25s 203ms/step - loss: 0.3552 - accuracy: 0.8494
Epoch 4/10
125/125 [==============================] - 25s 203ms/step - loss: 0.1815 - accuracy: 0.9365
Epoch 5/10
125/125 [==============================] - 25s 203ms/step - loss: 0.0948 - accuracy: 0.9682
Epoch 6/10
125/125 [==============================] - 25s 203ms/step - loss: 0.0597 - accuracy: 0.9812
Epoch 7/10
125/125 [==============================] - 25s 203ms/step - loss: 0.0211 - accuracy: 0.9919
Epoch 8/10
125/125 [==============================] - 26s 205ms/step - loss: 0.0057 - accuracy: 0.9989
Epo

In [66]:
# Create some predictions
predicted_y = clf.predict(X_test)

# Evaluate the model used
print(clf.evaluate(X_test, y_test, verbose = 1))

32/32 [==============================] - 2s 53ms/step - loss: 0.7898 - accuracy: 0.8310
[0.7898448705673218, 0.8309999704360962]


In [67]:
best_model = clf.export_model()

# Naming convenction = ModelAutoKeras_epochs_dataNo_trials
try:
    best_model.save("/content/drive/MyDrive/CIT/FYP/ImplementationFiles/ExportedModels/AutoKeras/Models/mak_10_5000_2", save_format="tf")
except:
    best_model.save("/content/drive/MyDrive/CIT/FYP/ImplementationFiles/ExportedModels/AutoKeras/Models/mak_10_5000_2.h5")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/CIT/FYP/ImplementationFiles/ExportedModels/AutoKeras/Models/mak_10_5000_2/assets


In [68]:
# Test loading a model
import tensorflow as tf
import tensorflow.keras

In [69]:
from tensorflow.keras.models import load_model
loaded_model = load_model("/content/drive/MyDrive/CIT/FYP/ImplementationFiles/ExportedModels/AutoKeras/Models/mak_100_5000_2", custom_objects=ak.CUSTOM_OBJECTS)

In [71]:
predicted_y = loaded_model.predict(tf.expand_dims(X_test, -1))
# print(predicted_y)
print(loaded_model.evaluate(X_test, y_test, verbose = 1))

32/32 [==============================] - 2s 53ms/step - loss: 0.0870 - accuracy: 0.9790
[0.08698087185621262, 0.9789999723434448]
